# **Converting 3D World Coord (from Blender) to 2D pixel coord of Local Camera**
Math concepts from https://www.scratchapixel.com/lessons/3d-basic-rendering/computing-pixel-coordinates-of-3d-point/mathematics-computing-2d-coordinates-of-3d-points.html

In [139]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
%matplotlib inline

# Reading Data from Textfiles imported from Blender

In [140]:
# grabs Euler rotation values from txt file, returns array [angle x, angle y, angle z]
def grabRotationValues(txtfile):
  # read in file's content
  file = open(txtfile, "r")
  contentArr = []

  while True:
    content = file.readline()
    contentArr.append(content.split("#"))
    if not content:
      break

  rMatrix = []
  # split 3 x,y,z values for rotation
  for i in range(1, len(contentArr) - 1):
    rMatrix.append((contentArr[i][2].split(",")))

  for j in range(len(rMatrix)):
    for m in range(3):
      rMatrix[j][m] = float(rMatrix[j][m])

  return rMatrix

In [141]:
# grabs camera translation from txtfile and returns [x,y,z]
def makeCameraTranslation(txtfile):
  # read in file's content
  file = open(txtfile, "r")
  contentArr = []

  while True:
    content = file.readline()
    contentArr.append(content.split("#"))
    if not content:
      break

  # make Camera Translation matrix
  camTranslation = []

  # split 3 x,y,z values for translation & rotation
  for i in range(1, len(contentArr) - 1):
    camTranslation.append((contentArr[i][1].split(",")))

  for j in range(len(camTranslation)):
    for m in range(3):
      camTranslation[j][m] = float(camTranslation[j][m])

  return camTranslation

# Main Function -- creates worldToCamera Matrix + cameraToWorld
Returns pScreen, NON-normalized 2D coordinate in camera view;
Returns pNormalized, Normalized 2D coordinate in camera view

In [142]:
def worldToCam(rotationMatrix, cameraTranslation, pWorld):
  cameraToWorld = np.zeros((4,4))
  lastColumn = [0,0,0,1]

  for i in range(3):
    for j in range(3):
        cameraToWorld[i][j] = rotationMatrix[i][j]
    cameraToWorld[3][i] = cameraTranslation[i]

  for m in range(4):
      cameraToWorld[m][3] = lastColumn[m]

  worldToCamera = np.linalg.inv(cameraToWorld)

  # pCamera = np.matmul(worldToCamera, np.transpose(pWorld))
  pCamera = np.matmul(pWorld, worldToCamera)

  pScreen = [0,0]
  pNormalized = [0,0]

  pScreen = [pCamera[0] / -pCamera[2], pCamera[1] / -pCamera[2]]

  # <-maxValue--------0---------maxValue->
  # 2 * maxValue = width of screen = 1080
  # 1080/(2 * maxValue) = pixel/blenderUnit
  # multiply by pScreen (blenderUnit), add offset of 540px

  maxValue = (540*0.3/460)

  pNormalized[0] = math.floor(540 + pScreen[0] * 1080/(2*maxValue))
  pNormalized[1] = math.floor(540 + pScreen[1] * 1080/(2*maxValue))

  return (pScreen, pNormalized)

Converts Euler angles from Blender into 3x3 rotation matrix

In [143]:
# this function converts one array [x-angle, y-angle, z-angle] to its corresponding rotation matrix
def eulerToRotation(x, y, z):
  """ x is the Euler x-angle, often notated as psi
      y is the Euler y-angle, often notated as theta
      z is the Euler z-angle, often notated as phi
      all angles are in radians
      math from https://eecs.qmul.ac.uk/~gslabaugh/publications/euler.pdf
  """
  # import math
  # import numpy as np

  # Rx = [[1, 0, 0],
  #       [0, math.cos(x), -math.sin(x)],
  #       [0, math.sin(x), math.cos(x)]]

  # Ry = [[math.cos(y), 0, math.sin(y)],
  #       [0, 1, 0],
  #       [-math.sin(y), 0, math.cos(y)]]

  # Rz = [[math.cos(z), -math.sin(z), 0],
  #       [math.sin(z), math.cos(z), 0],
  #       [0, 0, 1]]

  # rotMatrix = np.matmul((np.matmul(Rx, Ry)), Rz)

  # Scipy library https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.from_euler.html
  from scipy.spatial.transform import Rotation as R
  rotMatrix = R.from_euler('xyz', [x,y,z])

  return rotMatrix.as_matrix()

In [144]:
def createRotMatrix(eulerAngleMatrix):
  resultRotMatrix = []
  for i in eulerAngleMatrix:
    resultRotMatrix.append(eulerToRotation(i[0], i[1], i[2]))
  return resultRotMatrix

In [145]:
def main(file_name, object_file, num_frame):
  filename = file_name
  object_filename = object_file
  num_frames = num_frame

  # camTranslation format [[x,y,z], [x,y,z], ...]
  camTranslation = makeCameraTranslation(filename)

  # eulerRotation format in Euler RADIANS [[x-angle, y-angle, z-angle], ...]
  eulerRotation = grabRotationValues(filename)

  # create rotation matrix for every frame
  rotMatrix = createRotMatrix(eulerRotation)

  # pWorld is the coordinate of main object in world view
  pWorld = makeCameraTranslation(object_filename)

  # pScreen should be array of [[x,y], [x,y], ...]
  pScreen = []
  pNormalized = []
  for i in range(num_frames):
    pWorld_i = pWorld[i] + [1]
    pScreen.append(worldToCam(rotMatrix[i], camTranslation[i], pWorld_i)[0])
    pNormalized.append(worldToCam(rotMatrix[i], camTranslation[i], pWorld_i)[1])

  print(filename, "\n", pNormalized, "\n")
  # print(pScreen)

In [146]:
main("camera_horiz_position.txt", "object_position.txt", 100)     # ball right -> left
main("camera_position.txt", "object_position.txt", 100)           # ball down -> up
main("off_camera_position.txt", "object_position.txt", 100)       # off center ball bottom -> up
main("left_right_offcenter_cam.txt", "object_position.txt", 100)  # off center ball left -> right

main("box_updown_cam.txt", "object_position.txt", 100)            # off center cube down -> up
main("box_up_down.txt", "object_position.txt", 100)            # off center cube down -> up


# main("diag_camera_position.txt", "object_position.txt", 100)
# main("sword_camera_position.txt", "sword_position.txt", 100)
# main("sword_diagonal_position.txt", "sword_position.txt", 100)


camera_horiz_position.txt 
 [[1000, 540], [990, 540], [981, 540], [972, 540], [963, 540], [954, 540], [944, 540], [935, 540], [926, 540], [917, 540], [908, 540], [898, 540], [889, 540], [880, 540], [871, 540], [862, 540], [852, 540], [843, 540], [834, 540], [825, 540], [816, 540], [806, 540], [797, 540], [788, 540], [779, 540], [770, 540], [760, 540], [751, 540], [742, 540], [733, 540], [724, 540], [714, 540], [705, 540], [696, 540], [687, 540], [678, 540], [668, 540], [659, 540], [650, 540], [641, 540], [632, 540], [622, 540], [613, 540], [604, 540], [595, 540], [586, 540], [576, 540], [567, 540], [558, 540], [549, 540], [540, 540], [530, 540], [521, 540], [512, 540], [503, 540], [494, 540], [484, 540], [475, 540], [466, 540], [457, 540], [448, 540], [438, 540], [429, 540], [420, 540], [411, 540], [402, 540], [392, 540], [383, 540], [374, 540], [365, 540], [356, 540], [346, 540], [337, 540], [328, 540], [319, 540], [310, 540], [300, 540], [291, 540], [282, 540], [273, 540], [264, 540]